In [ ]:
import os
#import mlflow
import argparse
import sys
import logging

import datasets
from datasets import load_dataset
from peft import LoraConfig
import torch
import transformers
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset


In [ ]:
from huggingface_hub import login

login(token="")

In [ ]:
train_dataset = load_dataset("tryhighlight/task_dataset", split='train')

train_dataset

In [ ]:
train_dataset = train_dataset.select(range(1000))

In [ ]:
train_dataset.to_json(f"data/train.jsonl")

In [ ]:
test_dataset = load_dataset("tryhighlight/task_dataset", split='test')

test_dataset

In [ ]:
test_dataset = test_dataset.select(range(100))

In [ ]:
test_dataset.to_json(f"data/eval.jsonl")

In [ ]:
logger = logging.getLogger(__name__)


###################
# Hyper-parameters
###################
training_config = {
    "bf16": True,
    "do_eval": False,
    "learning_rate": 5.0e-06,
    "log_level": "info",
    "logging_steps": 20,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 5,
    "max_steps": -1,
    "output_dir": "./checkpoint_dir",
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 1,
    "per_device_train_batch_size": 1,
    "remove_unused_columns": True,
    "save_steps": 100,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs":{"use_reentrant": False},
    "gradient_accumulation_steps": 1,
    "warmup_ratio": 0.2,
    }


In [ ]:
peft_config = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": "all-linear",
    "modules_to_save": None,
}
train_conf = TrainingArguments(**training_config)
peft_conf = LoraConfig(**peft_config)

In [ ]:
###############
# Setup logging
###############
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = train_conf.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process a small summary
logger.warning(
    f"Process rank: {train_conf.local_rank}, device: {train_conf.device}, n_gpu: {train_conf.n_gpu}"
    + f" distributed training: {bool(train_conf.local_rank != -1)}, 16-bits training: {train_conf.fp16}"
)
logger.info(f"Training/evaluation parameters {train_conf}")

In [ ]:
################
# Modle Loading
################
checkpoint_path = "microsoft/Phi-3-mini-4k-instruct"
# checkpoint_path = "microsoft/Phi-3-mini-128k-instruct"
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    attn_implementation="flash_attention_2",  # loading the model with flash-attenstion support
    torch_dtype=torch.bfloat16,
    device_map=None
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 1024

In [ ]:
tokenizer.special_tokens_map

In [ ]:
tokenizer.pad_token

In [ ]:
tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

In [ ]:
system_prompt = "You are a helpful ai assistant. User will provide full name followed by some email or messaging conversation seen on his/her computer screen. Looking at the conversation, detect if there are any TODOs that the above user has to complete as a result of the conversation. If yes, just provide the short single line task that can be directly added to the todo list. If there is no task detected as a TODO, just output the exact phrase \"No task\". If the conversation is about a promotional or advertisement related, please output \"No task\". If the conversation is directed or addressed to someone else, then output \"No task\"."

In [ ]:
##################
# Data Processing
##################
def apply_chat_template(
    example,
    tokenizer,
):
    row_json = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "My name is " + example["NAME"] + " \n" + example["CONVERSATION"]},
        {"role": "assistant", "content": example["TASK"]}]
    example["text"] = tokenizer.apply_chat_template(
        row_json, tokenize=False, add_generation_prompt=False)
    return example

In [ ]:
train_dataset = load_dataset('json', data_files='data/train.jsonl', split='train')
test_dataset = load_dataset('json', data_files='data/eval.jsonl', split='train')
column_names = list(train_dataset.features)

In [ ]:
def check_length(row):
    # This function checks if the tokenized length is within the max length allowed
    input_content = tokenizer.encode(system_prompt + row["NAME"] + row["CONVERSATION"] + row["TASK"],
                                        add_special_tokens=True,
                                        truncation=False,
                                        return_length=True,
                                        max_length=None)
    return len(input_content) <= tokenizer.model_max_length - 20 # extra 20 tokens for the chat template

# Filter the dataset to exclude entries that are too long
train_dataset = train_dataset.filter(check_length)

In [ ]:
train_dataset

In [ ]:
test_dataset = test_dataset.filter(check_length)

In [ ]:
test_dataset

In [ ]:
processed_train_dataset = train_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    remove_columns=column_names,
    desc="Applying chat template to train_sft",
)

In [ ]:
processed_test_dataset = test_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    remove_columns=column_names,
    desc="Applying chat template to test_sft",
)

In [ ]:
processed_train_dataset

In [ ]:
processed_test_dataset

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=train_conf,
    peft_config=peft_conf,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_test_dataset,
    max_seq_length=tokenizer.model_max_length,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=True
)

In [ ]:
train_result = trainer.train()

In [ ]:
del trainer

In [ ]:
del model